In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import generate_objects, BunchTracker, track, compare_traces, compare_projections, Machine

from PyHEADTAIL.feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL.feedback.processors import ChargeWeighter, Averager, Bypass
from PyHEADTAIL.feedback.digital_processors import HilbertPhaseShiftRegister

np.random.seed(0)

In [ ]:
""" 
    In this example a more complex feedback system, which is an ideal bunch feedback but consists of 
    a separated pickup and a kicker is presented. More details from the ideal bunch feedback can be found
    from previous example (001_ideal_feedbacks.ipynb).

    @author Jani Komppula
    @date 26/09/2016
    @copyright CERN
"""

In [ ]:
# Basic parameters for the simulation
n_macroparticles = 10000
n_slices = 20
n_segments = 5
n_sigma_z = 3 

n_turns = 400

# Longitudinal motion of the bunch has not been taken into account in this example.
machine = Machine(Q_s = 0.00000001)
# If you want to include the longitudinal motion, please uncomment the following line
# machine = Machine(Q_s = 0.0020443)

bunch_ref, slicer_ref,trans_map, long_map = generate_objects(machine, n_macroparticles,n_segments, 
                                                             n_slices,n_sigma_z)
bunch_unkicked = copy.deepcopy(bunch_ref)


In [ ]:
# This creates an artificially kicked bunch, which will be damped with different feedback systems

slice_set = bunch_ref.get_slices(slicer_ref, statistics=True)
p_idx = slice_set.particles_within_cuts
s_idx = slice_set.slice_index_of_particle.take(p_idx)

# random kicks
kick_x = 0.003*(-1.0+2*np.random.rand(n_slices))
kick_y = 0.003*(-1.0+2*np.random.rand(n_slices))

for p_id, s_id in itertools.izip(p_idx,s_idx):
    bunch_ref.x[p_id] += kick_x[s_id]
    bunch_ref.y[p_id] += kick_y[s_id]
    
bunch_init = copy.deepcopy(bunch_ref)
tracker_ref = BunchTracker(bunch_init)
maps_ref = [i for i in trans_map] + [long_map]
track(n_turns, bunch_init,maps_ref,tracker_ref)

In [ ]:
# In the used code, the parameter 'gain' is determined as a fraction of the input signal if the used signal 
# processors would bypass the signalperfectly

# There are two ways to set a gain value. If only one value is given, same value will be used both
# in horizontal and vertical plane. If two values are given, separed values (x,y) are used for horizontal and
# vertical planes

feedback_gain = 0.01
# feedback_gain = (0.01,0.01)

# Parameters for the registers
delay = 1 
n_values = 3

In [ ]:
# A reference feedback system for the examples presented in this file is the ideal bunch feedback:

bunch_OneBox_bunch = copy.deepcopy(bunch_ref)
tracker_OneBox_bunch = BunchTracker(bunch_OneBox_bunch)
slicer_OneBox_bunch = copy.deepcopy(slicer_ref)

processors_bunch_x = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager()
]
processors_bunch_y = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager()
]

feedback_map = OneboxFeedback(feedback_gain,slicer_OneBox_bunch,processors_bunch_x,processors_bunch_y)
total_map_OneBox_bunch = [i for i in trans_map] + [long_map] + [feedback_map]

track(n_turns, bunch_OneBox_bunch,total_map_OneBox_bunch,tracker_OneBox_bunch)

# That feedback system is extended to consist of a separated pickup and a kicker in this file. If you don't
# understand details of the code above, please study examples from the file 001_ideal_feedbacks.ipynb

In [ ]:
# Before implementing a separated pickup and kicker, a new signal processors must be introduced; a register. 
# Registers is a signal processors which can store and delay a signal in the units of turns. In order to do 
# that they are also able to rotate the signal in terms of betatron motion.
#
# At the moment of writing this example, three different types of registers have been implemented 
# (VectorSumRegister(...), CosineSumRegister(...) and HilbertPhaseShiftRegister(...)). They work
# exactly similarly as a part of the list of signal processors, but different approaches for betatron phase 
# shift are utilized in them
#
# HilbertPhaseShiftRegister is used in the examples in this file for demonstrating the usage of a register 
# as a signal processor. At first, one turn delay is implemented by adding a register to the lists of signal
# processors

bunch_register_example = copy.deepcopy(bunch_ref)
tracker_register_example = BunchTracker(bunch_register_example)
slicer_register_example = copy.deepcopy(slicer_ref)


# One turn delay is implemented simply by adding a register to the lists of signal processors. Delay 
# (in the units of turns) is given as an input parameter for registers. The other required input parameters 
# for are a number of values stored to register (used for calculating the correct betatron phase for 
# the signal), and a tune of the machine in the studied plane.

processors_register_example_x = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_x, delay)
]
processors_register_example_y = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_y, delay)
]

feedback_map = OneboxFeedback(feedback_gain,slicer_register_example,
                              processors_register_example_x,processors_register_example_y)
total_map_register_example = [i for i in trans_map] + [long_map] + [feedback_map]

track(n_turns, bunch_register_example,total_map_register_example,tracker_register_example)


In [ ]:
# As it was explained, registers are used for storing the signal and rotating it in the betatron phase. Thus,
# a signal recorded in one place of the accelerator (e.g. pickup) can be rotated to the correct betatron 
# phase in the other place of the accelerator (e.g. kicker) by using a register
#
# A feedback system consisting of a separated pickup and kicker can be implemented similarly to the case
# of OneboxFeedback. Instead of adding OneboxFeedback object to the total tracking map of PyHEADTAIL, 
# separated Pickup and Kicker objects are added to the correct locations. The signal in both the pickup 
# and the kicker is processed by giving a list of signal processors as an input parameter to the objects. 
# The signal from the pickup is transferred to the kicker by giving a reference to the register, which is 
# the last element of the signal processors in the pickup, as an input parameter to the kicker. 

bunch_separated_example = copy.deepcopy(bunch_ref)
tracker_separated_example = BunchTracker(bunch_separated_example)
slicer_separated_example = copy.deepcopy(slicer_ref)

# At first, the pickup is created. It uses the exactly same signal processors as used in the previous example.
processors_separated_pickup_x = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_x, delay)
]
processors_separated_pickup_y = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager(),
    HilbertPhaseShiftRegister(n_values, machine.Q_y, delay)
]

# A location of the pickup in units of betatron phase advance, must be given as a input parameter for 
# Pickup(...) object. The total betatron phase angle over the accelerator is 2*pi*Q. In this example, 
# the accelerator is divided into a number of equally length segments determined by 'n_segments'. Thus, 
# the length of the one segment in betatron phase is 2*pi*Q/n_segments. The location of the pickup is after 
# the first segment, so this number is multiplied by one, i.e. 
pickup_location_x = 1.*2.*pi/float(n_segments)*machine.Q_x
pickup_location_y = 1.*2.*pi/float(n_segments)*machine.Q_y

# The map element is created by giving a slicer object and the parameters defined above as the input parameters
pickup_map = PickUp(slicer_separated_example,processors_separated_pickup_x,processors_separated_pickup_y, 
       pickup_location_x, pickup_location_y)




# The kicker can be created very similarly to the pickup. In addition to the input parameters given to PickUp,
# a gain value, references to the registers in the pickup and conversion 
# coefficients for amplitude conversion from displacement to divergence must be given as input parameters.

# In this example, signal processing in the kicker is not needed, so Bypass() processors are used
processors_separated_kicker_x = [Bypass()]
processors_separated_kicker_y = [Bypass()]

# In here a list of references to the register processors(s) in the pickup(s) is created
registers_x = [processors_separated_pickup_x[2]]
registers_y = [processors_separated_pickup_y[2]]

# In this example, the location of the kicker is after the second map element of of the trans_map.
# In betatron phase advance it means that
kicker_location_x = 2.*2.*pi/float(n_segments)*machine.Q_x
kicker_location_y = 2.*2.*pi/float(n_segments)*machine.Q_y

# The signal units in a pickup and a kicker are different. The pickup measures displacement in the units of 
# distance [m], but the kick is given in the kicker in the units of divergence [radian]. Thus, conversion
# coefficients from distance in the kicker to divergence in the kicker must br given. 
# The coefficients can be calculated, for example, from the ratios of the sigma values
xp_per_x = bunch_unkicked.sigma_xp()/bunch_unkicked.sigma_x()
yp_per_y = bunch_unkicked.sigma_yp()/bunch_unkicked.sigma_y()

kicker_map = Kicker(feedback_gain, slicer_separated_example,
                    processors_separated_kicker_x, processors_separated_kicker_y,
                    kicker_location_x, kicker_location_y,
                    registers_x, registers_y, xp_per_x, yp_per_y)

# After that the maps must be added to correct slots of the total map determined by the location of 
# the picup and the kicker
total_map_separated_example = [trans_map[0]] + [pickup_map] + [trans_map[1]] + [kicker_map]
for element in trans_map[2:]:
    total_map_separated_example += [element]
total_map_separated_example += [long_map]
    
    
track(n_turns, bunch_separated_example,total_map_separated_example,tracker_separated_example)

In [ ]:
# In here, the traces and the projections from different implementations of the feedback system are compared.
# Note the scale in the emittance figures.

compare_traces([tracker_OneBox_bunch,tracker_register_example,tracker_separated_example],
               ['Ideal', 'Delayed', 'Separated'])
compare_projections([ bunch_OneBox_bunch,  bunch_register_example, bunch_separated_example], 
                    ['Ideal', 'Delayed', 'Separated'])